In [1]:
import pandas as pd
import urllib
import numpy as np
import sklearn.linear_model as lm
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 14, 4

# <center>Tarea 2 Máquinas de Aprendizaje</center>

<center>
Patricio Horth M.<br>
Víctor Zúñiga M.<br>

2 de Noviembre de 2017
</center>



# 2. Análisis de Opiniones sobre Películas

El análisis de sentimiento (o minerı́a de opiniones) se refiere al proceso de extraer información acerca de
la actitud que una persona (o grupo de ellas) manifiesta, en un determinado medio o formato digital, con
respecto a un tópico o contexto de comunicación. Uno de los casos más estudiados corresponde a determinar
la polaridad de un trozo de texto, es decir, clasificar una determinada evaluación escrita (ó review ), en que
una persona manifiesta una opinión, como positiva, negativa o neutral. La dificultad de este problema radica
en el carácter altamente ambiguo e informal del lenguaje que utilizan naturalmente las personas ası́ como el
manejo de negaciones, sarcasmo y abreviaciones en una frase.

### a . Construya un dataframe con los datos a analizar descargando los datos desde la URL local. Determine cuántos registros de cada clase contiene el conjunto de entrenamiento y cuántos el conjunto de pruebas

In [48]:
#Se contruye el data frame 
train_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.train"
test_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.dev"
train_data_f = urllib.request.urlretrieve(train_data_url, "train_data_item2.csv")
test_data_f = urllib.request.urlretrieve(test_data_url, "test_data_item2.csv")
ftr = open("train_data_item2.csv", "r")
fts = open("test_data_item2.csv", "r")
rows = [line.split(" ",1) for line in ftr.readlines()]
train_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
train_df['Sentiment'] = pd.to_numeric(train_df['Sentiment'])
rows = [line.split(" ",1) for line in fts.readlines()]
test_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
test_df['Sentiment'] = pd.to_numeric(test_df['Sentiment'])

Para este dataset tenemos que la cantidad de clases estará dada por la columna "Sentiment" del dataset, donde existen dos valores posibles $+1$ para comentarios positivos y $-1$ para comentarios negativos.

In [51]:
test_df.head()

,Sentiment,Text
0,-1,screenwriter dan schneider and director shawn ...
1,-1,"home alone goes hollywood , a funny premise un..."
2,1,seldom has a movie so closely matched the spir...
3,-1,"less dizzying than just dizzy , the jaunt is p..."
4,-1,an ultra-low-budget indie debut that smacks mo...


Pr otro lado, la cantidad de registros totales que incluye cada data set será:

In [65]:
count_train_df = pd.value_counts(train_df['Sentiment'].values, sort=False)
count_test_df = pd.value_counts(test_df['Sentiment'].values, sort=False)
print("================Conjunto de entrenamiento================")
print("La cantidad de elementos que tienen la clase +1 es: ",count_train_df[1])
print("La cantidad de elementos que tienen la clase -1 es: ",count_train_df[-1])
print("Total:",train_df.shape[0], "registros")
print("=========================================================")
print("================Conjunto de prueba=======================")
print("La cantidad de elementos que tienen la clase +1 es: ",count_test_df[1])
print("La cantidad de elementos que tienen la clase -1 es: ",count_test_df[-1])
print("Total:",test_df.shape[0], "registros")
print("=========================================================")

================Conjunto de entrenamiento================
La cantidad de elementos que tienen la clase +1 es:  1770
La cantidad de elementos que tienen la clase -1 es:  1784
Total: 3554 registros
================Conjunto de prueba=======================
La cantidad de elementos que tienen la clase +1 es:  1751
La cantidad de elementos que tienen la clase -1 es:  1803
Total: 3554 registros


Como podemos apreciar en nuestros dataset, existe un pequeño desbalance en la cantidad de registros que tiene cada clase, ya que para ambos casos (entrenamiento y test) la clase $-1$ tiene más registros, pero como la diferencia es practicamente despreciable supondremos que no representará un problema a la hora de clasificar los datos.

### (b) Construya una función, denominada word extractor, que devuelva una lista de las palabras contenidas en un determinado un trozo de texto. Incorpore en su función las operaciones de lower-casing y stemming. Pruebe la función con las frases sugeridas en el código, invente otras similares y comente. Compare con los resultados obtenidos si no se hace stemming.

A continuacion se implementa la funcion word_extractor, la cual hace un preprocesamiento de las frases contenidas en la columna Text del dataset, la cual quita las stop words y aplica stemming mediante el algoritmo de Porter. Tambien  se encuentra la implementacion de la funcion word_extractor_simple que a diferencia de la anteriorel unico preporcesamiento que realiza es simplemente quitar las stop words de cada frase.
Al aplicar stemming a una palabra lo que se hace es encontrar su raiz, es decir, se busca que la forma de las palabras no penalice la frecuencia de ésta ya que, una palabra puede estar conjugada en cualquier (género, número, persona, etc.) y solo se considerará (en muchas ocasiones) como un solo término.

In [78]:
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
from nltk.stem.porter import PorterStemmer

def word_extractor(text,include_stopwords = False):
    ps = PorterStemmer()
    text = re.sub(r'([a-z])\1+', r'\1\1',text)
    words = ""
    wordtokens = [ ps.stem(word.lower()) for word in word_tokenize(text) ]
    if include_stopwords == False:
        commonwords = stopwords.words('english')
        commonwords.remove("not")
        for word in wordtokens:
            if word not in commonwords:
                words+=" "+word
    else:
        for word in wordtokens:
            words+=" "+word
    return words

def word_extractor_simple(text):
    ps = PorterStemmer()
    #Se eliminan las "palabras vacias"
    commonwords = stopwords.words('english')
    commonwords.remove("not")
    text = re.sub(r'([a-z])\1+', r'\1\1',text)
    words = ""
    wordtokens = [ word.lower() for word in word_tokenize(text) ]
    for word in wordtokens:
        #Se dejan solo las palabras relevantes
        if word not in commonwords:
            words+=" "+word
    return words                   

A contuacion se probara el uso de ambas funciones con algunas palabras de prueba.

In [79]:
print("===============================")
print("Pruebas usando Porter Stemmer ")
print("===============================")
print(word_extractor("I love to eat cake",False))
print(word_extractor("I love eating cake",False))
print(word_extractor("I loved eating the cake",False))
print(word_extractor("I do not love eating cake",False))
print(word_extractor("I don't love eating cake",False))
print(word_extractor("You awaked late",False))
print(word_extractor("You awoke late",False))
print(word_extractor("You have been awaking late",False))
print(word_extractor("I would have awaken",False))
print("===============================")
print("Pruebas solo quitando stop words")
print("===============================")
print(word_extractor_simple("I love to eat cake"))
print(word_extractor_simple("I love eating cake"))
print(word_extractor_simple("I loved eating the cake"))
print(word_extractor_simple("I do not love eating cake"))
print(word_extractor_simple("I don't love eating cake"))
print(word_extractor_simple("You awoke late"))
print(word_extractor_simple("You have been awaking late"))
print(word_extractor_simple("I would have awaken"))

Pruebas usando Porter Stemmer 
 love eat cake
 love eat cake
 love eat cake
 not love eat cake
 n't love eat cake
 awak late
 awok late
 awak late
 would awaken
Pruebas solo quitando stop words
 love eat cake
 love eating cake
 loved eating cake
 not love eating cake
 n't love eating cake
 awoke late
 awaking late
 would awaken


Como podemos apreciar la diferencia es evidente ya que al usar en el pre procesamiento Porter Stemmer al quedar solo la raiz de la palabra, la frecuencia de esta aumenta, mientras que al solo quitar las stop words las conjugaciones de cada palabra se mantienen y serán consideradas como una palabra diferente.

### (c) Construya una función, denominada word extractor2, análoga a la función anterior, pero que lematice las palabras en vez de hacer stemming. Pruebe la función con las frases sugeridas en el código anterior y discuta las diferencias que observa.


En esta sección se presentara la implementacion de la funcion word_extractor2 para realizar el preprocesamiento de las palabras contenidas en el dataset. La diferencia de esta funcion con las demás radica en que se sustituira el uso de Stemming por un algoritmo de lematización.

https://stackoverflow.com/questions/25534214/nltk-wordnet-lemmatizer-shouldnt-it-lemmatize-all-inflections-of-a-word

In [23]:
import nltk
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\horth_000\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [71]:
# Correción aportada por Francisco Casas en el moodle del curso
# https://moodle.inf.utfsm.cl/mod/forum/discuss.php?d=31466

def get_wordnet_tag(tag):
    if tag.startswith('JJ'):
        return 'a'
    elif tag.startswith('RB') or tag == "WRB":
        return 'r'
    elif tag.startswith('NN') or tag.startswith("WP"):
        return 'n'
    elif tag.startswith('VB'):
        return 'v'
    else:
        return None

def lemmatize_words(lemmatizer,words):
    tagged = pos_tag(words)
    lemmas = []
    for word, tag in tagged:
        wntag = get_wordnet_tag(tag)
        if wntag is None:
            lemma = lemmatizer.lemmatize(word)
        else:
            lemma = lemmatizer.lemmatize(word,pos=wntag)
        lemmas.append(lemma)
    return lemmas

In [80]:
def word_extractor2(text,include_stopwords = False):
    wordlemmatizer = WordNetLemmatizer()
    text = re.sub(r'([a-z])\1+', r'\1\1',text)
    words = ""
    wordtokens = lemmatize_words(wordlemmatizer,word_tokenize(text.lower()))
    if include_stopwords == False:
        commonwords = stopwords.words('english')
        commonwords.remove("not")
        for word in wordtokens:
            if word not in commonwords:
                words+=" "+word
    else:
        for word in wordtokens:
            words+=" "+word
    return words

La lematización es un proceso lingüístico que consiste en, dada una forma flexionada (es decir, en plural, en femenino, conjugada, etc), se encuentra el lema correspondiente. El lema es la forma que por convenio se acepta como representante de todas las formas flexionadas de una misma palabra. Es decir, el lema de una palabra es la palabra que nos encontraríamos como entrada en un diccionario tradicional: singular para sustantivos, masculino singular para adjetivos, infinitivo para verbos.

In [81]:
print("===============================")
print("Pruebas usando lematización")
print("===============================")
print(word_extractor2("I love to eat cake",False))
print(word_extractor2("I love eating cake",False))
print(word_extractor2("I loved eating the cake",False))
print(word_extractor2("I do not love eating cake",False))
print(word_extractor2("I don't love eating cake",False))
print(word_extractor2("You awoke late",False))
print(word_extractor2("You have been awaking late",False))
print(word_extractor2("I would have awaken",True))


Pruebas usando lematización
 love eat cake
 love eat cake
 love eat cake
 not love eat cake
 n't love eat cake
 awake late
 awake late
 i would have awaken


Como podemos apreciar a diferencia del uso de un algoritmo de stemming que realiza cortes alfinal y en ocaciones al inicio de una palabra en busca de su raiz, donde este corte "indiscriminado" puede resultar ser correcto en ciertas ocaciones pero no siempre, la lematizacion hace un análisis morfologico de la palabra entregando de forma correcta el lemma de la palabra, es decir la base para todas las formas flexionadas de esta. Además la raíz o stemm puede ser la misma para las formas flexionadas de diferentes lemas, lo que proporciona ruido a nuestros resultados de búsqueda.

Comparando los resultados obtenidos en este punto con el anterior podemos verificar que el uso de la lematizacion es mas preciso que el uso de algún stemmer, por ejemplo para las conjugaciones del verbo awake el uso de stemming entrego dos raices distintas, mientras que la lematizacion entrego para todas sus conjugaciones el mismo lemma.   

#### (d) Utilizando la función CountVectorizer de la librería sklearn y de acuerdo a las directrices mencionadasen la introducción, genere una representación vectorial del texto de entrenamiento y del conjunto que usaremos para realizar pruebas. Explore el vocabulario utilizado y determine cuáles son las palabras más frecuentes en el conjunto de entrenamiento y pruebas.

In [82]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tabulate  import tabulate

def vect_rep(train_df, test_df, v = "porter", include_stopwords = False, v1 = "train"):
    if v == "porter":
        texts_train = [word_extractor(text,include_stopwords) for text in train_df.Text]
        texts_test = [word_extractor(text,include_stopwords) for text in test_df.Text]
    if v == "lem":
        texts_train = [word_extractor2(text,include_stopwords) for text in train_df.Text]
        texts_test = [word_extractor2(text,include_stopwords) for text in test_df.Text]
    if v == "simple":
        texts_train = [word_extractor_simple(text) for text in train_df.Text]
        texts_test = [word_extractor_simple(text) for text in test_df.Text]
    vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
    vectorizer.fit(np.asarray(texts_train))
    vocab = vectorizer.get_feature_names()
    if v1 == "train":
        features_train = vectorizer.transform(texts_train)
        labels_train = np.asarray((train_df.Sentiment.astype(float)+1)/2.0)
        dist=list(np.array(features_train.sum(axis=0)).reshape(-1,))
        features = features_train
        labels = labels_train
    if v1 == "test":
        features_test = vectorizer.transform(texts_test)
        labels_test = np.asarray((test_df.Sentiment.astype(float)+1)/2.0)
        dist=list(np.array(features_test.sum(axis=0)).reshape(-1,))
        features = features_test
        labels = labels_test
    #Se modifica el codigo original para guardar cada tag y su frecuencia 
    frec_tag = []
    for tag, count in zip(vocab, dist):
        frec_tag.append([count, tag])
    return frec_tag,features,labels 

In [27]:
def n_more_frequent(l,n):
    # Se ordenan de manera descendente
    sorted_tags = sorted(l, reverse=True)
    # Se guardan las n palabras mas frecuentes
    elements = []
    for i in range(n):
        elements.append(sorted_tags[i])
    return elements

def tab(a,b,c,n):
    aa = n_more_frequent(a,n)
    bb = n_more_frequent(b,n)
    cc = n_more_frequent(c,n)
    from tabulate import tabulate
    tab = []
    for i  in range(len(aa)):
        tab.append([aa[i],bb[i],cc[i]])
    return tab   

In [44]:
a,features_train_porter,labels_train_porter = vect_rep(train_df, test_df, "porter",False,"train")
b,features_train_lem,labels_train_lem = vect_rep(train_df, test_df, "lem",False,"train")
e,features_train_simple,labels_train_simple = vect_rep(train_df, test_df, "simple","train")

In [45]:
c,features_test_porter,labels_test_porter = vect_rep(train_df, test_df, "porter",False,"test")
d,features_test_lem,labels_test_lem = vect_rep(train_df, test_df, "lem",False,"test")
f,features_test_simple,labels_test_simple = vect_rep(train_df, test_df, "simple","test")

In [77]:
n = 15
tab_train = tab(a,b,e,n)
tab_test = tab(c,d,f,n)

print("Las",n,"palabras mas frecuentes para el conjunto de entrenamiento")
print(tabulate(tab_train, headers=['PorterStemmer', 'wordlemmatizer', 'Simple']))
print("\n")
print("Las",n,"palabras mas frecuentes para el conjunto de prueba")
print(tabulate(tab_test, headers=['PorterStemmer', 'wordlemmatizer','Simple']))

Las 15 palabras mas frecuentes para el conjunto de entrenamiento
PorterStemmer     wordlemmatizer      Simple
----------------  ------------------  -------------------
[575, 'film']     [574, 'film']       [519, 'film']
[477, 'movi']     [481, 'movie']      [421, 'movie']
[425, 'thi']      [266, 'not']        [266, 'not']
[269, 'not']      [264, 'make']       [244, 'like']
[268, 'like']     [254, 'like']       [238, 'one']
[245, 'one']      [246, 'one']        [158, 'story']
[224, 'ha']       [176, 'story']      [143, 'even']
[207, 'make']     [169, 'good']       [140, 'good']
[179, 'hi']       [163, 'character']  [135, 'comedy']
[176, 'stori']    [145, 'comedy']     [128, 'much']
[161, 'charact']  [143, 'time']       [113, 'characters']
[148, 'time']     [143, 'even']       [111, 'well']
[144, 'good']     [129, 'work']       [107, 'time']
[144, 'even']     [128, 'much']       [103, 'make']
[142, 'comedi']   [127, 'well']       [103, 'little']


Las 15 palabras mas frecuentes para el c

Como podemos para ambos datasets el uso del stemmer y la lematizacion entregan resultados muy similares, sin embargo podemos ver que el uso de la lematizaciónentrega algunas palabras que proveen un poco mas de información. Por otro lado solo quitar las palabras vacias se obtienen un resultado muy similar al uso de la lematizacion pero la frecuencia de cada palabra entregada es bastante menor. 

### e)

In [84]:
from sklearn.metrics import classification_report

def get_error(yhat, ytest):
    error_total = 0
    for i in range(1, len(yhat)):
        if yhat[i] != ytest[i]:
            error_total = error_total + 1
    return error_total / len(yhat)

def score_the_model(model,x,y,xt,yt,text):
    acc_tr = model.score(x,y)
    acc_test = model.score(xt[:-1],yt[:-1])
    error = get_error(model.predict(xt),yt)
    print("Training Accuracy %s: %f"%(text,acc_tr))
    print("Test Accuracy %s: %f"%(text,acc_test))
    print("Error:", error)
    print( "Detailed Analysis Testing Results ...")
    print(classification_report(yt, model.predict(xt), target_names=['+','-']))


http://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/

Tenemos que las metricas que entrega la funcion classification report son un resumen de texto de:
    
$Precision:$  La precisión es el ratio entre las observaciones positivas predichas correctamente y las observaciones positivas predichas totales.Su calculo esta dado por: 

$\displaystyle \frac{\text{verdadero postivo}}{\text{verdaro positivo} + \text{falso positivo}}$
    
$Recall:$ Esta metrica representa el ratio de observaciones positivas predichas correctamente del total de observaciones en la clase actual. Su calculo esta dado por: 

$\displaystyle \frac{\text{verdadero postivo}}{\text{verdaro positivo} + \text{falso negativo}}$

$F1 score:$ Se puede interpretar como un promedio ponderado de la precisión y el recall, donde alcanza su mejor valor en 1 y el peor puntaje en 0. La contribución relativa de la precisión y el recall son iguales. Su calculo esta dado por:

 $\displaystyle F1 = 2 * \frac{precision * recall}{precision + recall}$

#### (f) Construya una función que entrene/ajuste un clasificador Bayesiano Ingenuo (Binario) (las caracteristicas no nulas serán tratadas como 1) y mida el error de predicción obtenido sobre los datos de entrenamiento y pruebas. 
#### Utilice esta función con las características extraídas en el punto (d). Mida el efecto de filtrar stopwords y de eliminar este paso de pre-procesamiento típico. 
#### Determine además, qué representación obtiene un mejor resultado: si aquella obtenida vía lematización o aquella obtenida vía stemming. 
#### Finalmente, tome un subconjunto aleatorio de los textos de prueba y analice las predicciones del modelo (explore las predicciones, así como las probabilidades que el clasificador asigna a cada clase).


In [85]:
from sklearn.naive_bayes import BernoulliNB
import random

#Error de clasificacion

def do_NAIVE_BAYES(x,y,xt,yt):
    model = BernoulliNB()
    model = model.fit(x, y)
    score_the_model(model,x,y,xt,yt,"BernoulliNB")
    return model

def get_model_metrics(features_train,labels_train, features_test, labels_test):
    model=do_NAIVE_BAYES(features_train,labels_train,features_test,labels_test) 
    #Retorna la probabilidad de pertenencia para cada clase del modelo usando el vector de test.
    test_pred = model.predict_proba(features_test)
    #Selecciona de forma aleatorea la posicion de 15 elementos en el vector test_pred.
    spl = random.sample(range(len(test_pred)),10)
    data = []
    for text, sentiment in zip(test_df.Text[spl], test_pred[spl]):
        data.append([sentiment, text[0:80]])
    print(tabulate(data, headers=['Sentiment', 'Text'],tablefmt="fancy_grid"))

In [33]:
ftag_1,features_train_porter2,labels_train_porter2 = vect_rep(train_df, test_df, "porter",True,"train")
ftag_2,features_test_porter2,labels_test_porter2 = vect_rep(train_df, test_df, "porter",True,"test")

In [34]:
ftag_3,features_train_lem2,labels_train_lem2 = vect_rep(train_df, test_df, "lem",True,"train")
ftag_4,features_test_lem2,labels_test_lem2 = vect_rep(train_df, test_df, "lem",True,"test")

In [86]:
print("usando porter")
print("-------------------------------------------------------------------")
get_model_metrics(features_train_porter, labels_train_porter, features_test_porter,labels_test_porter)
print("\n")

usando porter
-------------------------------------------------------------------
Training Accuracy BernoulliNB: 0.942600
Test Accuracy BernoulliNB: 0.748100
Error: 0.2518289251547552
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.76      0.75      0.75      1803
          -       0.74      0.75      0.75      1751

avg / total       0.75      0.75      0.75      3554

╒═════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════╕
│ Sentiment                           │ Text                                                                             │
╞═════════════════════════════════════╪══════════════════════════════════════════════════════════════════════════════════╡
│ [ 0.77443158  0.22556842]           │ i found it slow , predictable and not very amusing .                             │
├─────────────────────────────────────┼────────────────────────────────

In [87]:
print("usando porter sin quitar las stopwords")
print("-------------------------------------------------------------------")
get_model_metrics(features_train_porter2, labels_train_porter2, features_test_porter2,labels_test_porter2)

usando porter sin quitar las stopwords
-------------------------------------------------------------------
Training Accuracy BernoulliNB: 0.938661
Test Accuracy BernoulliNB: 0.762736
Error: 0.23719752391671356
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.77      0.76      0.77      1803
          -       0.76      0.76      0.76      1751

avg / total       0.76      0.76      0.76      3554

╒═════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════╕
│ Sentiment                           │ Text                                                                             │
╞═════════════════════════════════════╪══════════════════════════════════════════════════════════════════════════════════╡
│ [ 0.01041238  0.98958762]           │ a smart and funny , albeit sometimes superficial , cautionary tale of a technolo │
├─────────────────────────────────────┼──────

Realizando una comparativa podemos ver que el desempeño en general del modelo aumenta si no quitamos las stop words.

In [88]:
print("usando lematizacion")
print("-------------------------------------------------------------------")
get_model_metrics(features_train_lem, labels_train_lem, features_test_lem,labels_test_lem)

usando lematizacion
-------------------------------------------------------------------
Training Accuracy BernoulliNB: 0.951604
Test Accuracy BernoulliNB: 0.742753
Error: 0.25717501406865506
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.75      0.74      0.75      1803
          -       0.74      0.74      0.74      1751

avg / total       0.74      0.74      0.74      3554

╒═══════════════════════════╤══════════════════════════════════════════════════════════════════════════════════╕
│ Sentiment                 │ Text                                                                             │
╞═══════════════════════════╪══════════════════════════════════════════════════════════════════════════════════╡
│ [ 0.44054005  0.55945995] │ if you can push on through the slow spots , you'll be rewarded with some fine ac │
├───────────────────────────┼───────────────────────────────────────────────────────────────────────────

In [89]:
print("usando lematizacion sin quitar las stopwords")
print("-------------------------------------------------------------------")
get_model_metrics(features_train_lem2, labels_train_lem2, features_test_lem2,labels_test_lem2)

usando lematizacion sin quitar las stopwords
-------------------------------------------------------------------
Training Accuracy BernoulliNB: 0.946539
Test Accuracy BernoulliNB: 0.750070
Error: 0.24985931344963422
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.75      0.75      0.75      1803
          -       0.75      0.75      0.75      1751

avg / total       0.75      0.75      0.75      3554

╒═════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════╕
│ Sentiment                           │ Text                                                                             │
╞═════════════════════════════════════╪══════════════════════════════════════════════════════════════════════════════════╡
│ [ 0.77022531  0.22977469]           │ this 72-minute film does have some exciting scenes , but it's a tad slow .       │
├─────────────────────────────────────┼

Como podemos apreciar al no quitar las stop words las metricas para el modelo tambien aumentan, sin embargo el uso de la lematizacion entrega un desempeño un poco menor que el uso de un stemmer esto posiblemente ya que el uso de algoritmos de stemming aunmentan el recall, y el f1 score esta en terminos del recall, por lo que si este aumenta esta metrica el f1 score tambien lo hará.

#### (g) Construya una función que entrene/ajuste una Máquina de Vectores de Soporte (SVM) Lineal y mida el error de predicción obtenido sobre los datos de entrenamiento y pruebas. 
#### * Incluya en su función la exploración de diferentes valores del parámetro de regularización C.
#### * Discuta el significado y efecto esperado de este parámetro. Utilice la función construida con los atributos extraídos en el punto (d). 
#### *  Mida el efecto de filtrar stopwords y de eliminar este paso de pre-procesamiento típico. Determine además, qué representación obtiene un mejor resultado: si aquella obtenida vía lematización o aquella obtenida vía stemming. 
* Finalmente, tome un subconjunto aleatorio de los textos de prueba y analice las predicciones del modelo (explore las predicciones, así como las probabilidades que el clasificador asigna a cada clase).

In [101]:
from sklearn.svm import LinearSVC

def do_SVM(x,y,xt,yt):
    Cs = [0.01,0.1,1,10,100,1000]
    for C in Cs:
        print("C Value:",C)
        model = LinearSVC(C=C)
        model = model.fit(x, y)
        print(model.score(xt,yt))
        score_the_model(model,x,y,xt,yt,"SVM")
        print("====================================================")
        
def do_SVM2(x,y,xt,yt,C):
        print("C Value:",C)
        model = LinearSVC(C=C)
        model = model.fit(x, y)
        return model
        
def get_random_set(features_train,labels_train, features_test, labels_test,C,n):
    model = do_SVM2(features_train,labels_train,features_test,labels_test,C)
    #Retorna la probabilidad de pertenencia para cada clase del modelo usando el vector de test.
    #test_pred = model.decision_function(features_test)
    test_pred = model.predict(features_test)
    #Selecciona de forma aleatorea la posicion de 15 elementos en el vector test_pred.
    spl = random.sample(range(len(test_pred)),n)
    data = []
    for text, sentiment in zip(test_df.Text[spl], test_pred[spl]):
        data.append([sentiment, text[0:85]])
    print(tabulate(data, headers=['Sentiment', 'Text'],tablefmt="fancy_grid"))


https://www.ibm.com/support/knowledgecenter/es/SS3RA7_18.0.0/modeler_mainhelp_client_ddita/clementine/svm_perfimp.html

Tenemos que el parametro C de la funcion LinearSVC, corresponde a un parametro de regularizacion, es decir, le dice a la SVM cuánto desea evitar la clasificación errónea de cada dato de entrenamiento. Tenemos que para valores grandes de C, se escogerá un hiperplano de margen más pequeño, esperando que ese hiperplano clasifique mejor los datos del conjunto de entrenamiento. Por el contrario, un valor muy pequeño de C hará que el optimizador busque un hiperplano de separación de mayor margen, incluso si ese hiperplano clasifica erróneamente más puntos. Todo lo anterior se traduce en encontrar el valor de C optimo que le de una mejor comportamiento a nuestro modelo. A continuación  se probará de varias SVM usando Stemming y lematizacion para varios valores de C y filtrando o no las Stop Words.

#### Pruebas

En esta parte se utilizará lematización y el algoritmo de porter considerando la elieminación de las stopwors o "palabras vacias", es decir, todas aquellas palabras que no son relevantes (ya que no poseen significado alguno y que por lo general son los conectores, pronombres, preposiciones y articulos presentes en una oración) como parte del pre-procesamiento de la data para nuestro clasificador. Además para poder determinar que modelo se comporta de mejor manera comprando los f1 score obtenidos a partir de los distintos valores de C.

##### Pruebas usando WordNetLemmatizer

In [90]:
do_SVM(features_train_lem,labels_train_lem, features_test_lem, labels_test_lem)

C Value: 0.01
0.710748452448
Training Accuracy SVM: 0.882667
Test Accuracy SVM: 0.710667
Error: 0.289251547552054
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.71      0.73      0.72      1803
          -       0.71      0.70      0.70      1751

avg / total       0.71      0.71      0.71      3554

C Value: 0.1
0.719752391671
Training Accuracy SVM: 0.987338
Test Accuracy SVM: 0.719674
Error: 0.28024760832864376
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.72      0.72      0.72      1803
          -       0.72      0.72      0.72      1751

avg / total       0.72      0.72      0.72      3554

C Value: 1
0.705402363534
Training Accuracy SVM: 0.999156
Test Accuracy SVM: 0.705319
Error: 0.2945976364659538
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.71      0.71      0.71      1803
        

Como podemos apreciar en las pruebas realizadas, los mejores indicadores se obtienen para un valor de $C = 0.1$, teniendo un training accuracy de 0.987338, un test accuracy de 0.719674 para la data de test y un f1 score de 0.72.

Para valores a partir de $C = 1$ el conjunto de entrenamiento tiene un accuray de 1, lo que hace que se produzca sobreajuste, viendose reflejado en que todos los indicadores a partir de este valor comienzan a decaer mientras más grande es el valor de C.

#### Pruebas usando PorterStemmer

Análogo a lo anterior se realizaran las pruebas sin quitar las stop words , pero esta vez usando stemming a traves del algoritmo de porter.


In [91]:
do_SVM(features_train_porter,labels_train_porter, features_test_porter, labels_test_porter)

C Value: 0.01
0.729319077096
Training Accuracy SVM: 0.872538
Test Accuracy SVM: 0.729243
Error: 0.2706809229037704
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.73      0.75      0.74      1803
          -       0.73      0.71      0.72      1751

avg / total       0.73      0.73      0.73      3554

C Value: 0.1
0.733539673607
Training Accuracy SVM: 0.981711
Test Accuracy SVM: 0.733465
Error: 0.2664603263927969
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.74      0.74      0.74      1803
          -       0.73      0.73      0.73      1751

avg / total       0.73      0.73      0.73      3554

C Value: 1
0.720596510974
Training Accuracy SVM: 0.999719
Test Accuracy SVM: 0.720518
Error: 0.27940348902644907
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.73      0.72      0.72      1803
       

Para este caso podemos ver que el valor de C que entrega un mejor resultado es $C = 0.1$. Al igual que para el caso, anterior podemos notar que a partir de valores $C = 1$ el training accuracy es practicamente 1, lo que hace que los indicadores para la data de test vaya disminuyendo.

#### Pruebas sin quitar las stopwords

En esta parte se utilizará lematización y el algoritmo de porter manteniendo las stopwors o "palabras vacias", en la parte del pre-procesamiento de la data para nuestro clasificador. Además al igual  que la parte anterior para poder determinar que modelo se comporta de mejor manera se comparará los f1 score obtenidos a partir de los distintos valores de C.

##### Pruebas usando WordNetLmmatizer

In [92]:
do_SVM(features_train_lem2,labels_train_lem2, features_test_lem2, labels_test_lem2)

C Value: 0.01
0.714124929657
Training Accuracy SVM: 0.870850
Test Accuracy SVM: 0.714044
Error: 0.2858750703432752
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.72      0.72      0.72      1803
          -       0.71      0.71      0.71      1751

avg / total       0.71      0.71      0.71      3554

C Value: 0.1
0.733258300506
Training Accuracy SVM: 0.986775
Test Accuracy SVM: 0.733183
Error: 0.26674169949352844
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.74      0.73      0.74      1803
          -       0.73      0.74      0.73      1751

avg / total       0.73      0.73      0.73      3554

C Value: 1
0.714406302757
Training Accuracy SVM: 1.000000
Test Accuracy SVM: 0.714326
Error: 0.28559369724254363
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.73      0.70      0.71      1803
      

Nuevamente encontramos que el mejor valor para el parametro de regularización será $C=0.1$ ya que se obtienen mejores metricas en comparación con las demás pruebas para los diferentes valores de este parametro. Además notamos que al mantener las palabras vacias las matricas en general aumentaron especialmente en los valores mayores a $C=0.01$ en comparación con el caso donde estas se eliminaban. 

#### Pruebas usando PorterStemmer

In [97]:
do_SVM(features_train_porter2,labels_train_porter2, features_test_porter2, labels_test_porter2)

C Value: 0.01
0.729881823298
Training Accuracy SVM: 0.868317
Test Accuracy SVM: 0.729806
Error: 0.2698368036015757
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.73      0.74      0.74      1803
          -       0.73      0.72      0.72      1751

avg / total       0.73      0.73      0.73      3554

C Value: 0.1
0.739729881823
Training Accuracy SVM: 0.982836
Test Accuracy SVM: 0.739657
Error: 0.2602701181767023
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.74      0.74      0.74      1803
          -       0.73      0.74      0.74      1751

avg / total       0.74      0.74      0.74      3554

C Value: 1
0.729319077096
Training Accuracy SVM: 1.000000
Test Accuracy SVM: 0.729243
Error: 0.2706809229037704
Detailed Analysis Testing Results ...
             precision    recall  f1-score   support

          +       0.74      0.72      0.73      1803
        

Nuevamente encontramos que el mejor valor para el parametro de regularización será $C=0.1$ ya que se obtienen mejores metricas en comparación con las demás pruebas para los diferentes valores de este parametro. Además notamos que al mantener las palabras vacias las matricas en general aumentaron $0.01$ en comparación con el caso donde estas se eliminaban. 

Finalmente al igual que en Naive Bayes, tenemos que al usar Stemming en el modelo se obtiene un mejor resultado, esto posiblemente a que se aumenta el "ruido" a la hora de realizar la clasificación dandole una mayor capacidad de generalizacion al modelo. Tambien logramos identificar en todas nuestras pruebas que el valor optimo para el parametro de regularizacion será $C = 0.1$ ya que al usarlo siempre y de forma generalizada se obtenia una mejora para todas metricas.

In [102]:
get_random_set(features_train_porter,labels_train_porter, features_test_porter, labels_test_porter,0.1,10)

C Value: 0.1
╒═════════════╤═══════════════════════════════════════════════════════════════════════════════════════╕
│   Sentiment │ Text                                                                                  │
╞═════════════╪═══════════════════════════════════════════════════════════════════════════════════════╡
│           1 │ it's endlessly inventive , consistently intelligent and sickeningly savage .          │
├─────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│           1 │ an escapist confection that's pure entertainment .                                    │
├─────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│           1 │ never quite transcends jokester status . . . and the punchline doesn't live up to bar │
├─────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│           1 │ the artwork is spectacular and unli

In [103]:
get_random_set(features_train_porter,labels_train_porter, features_test_porter, labels_test_porter,1000,10)

C Value: 1000
╒═════════════╤═══════════════════════════════════════════════════════════════════════════════════════╕
│   Sentiment │ Text                                                                                  │
╞═════════════╪═══════════════════════════════════════════════════════════════════════════════════════╡
│           0 │ demands too much of most viewers .                                                    │
├─────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│           1 │ an escapist confection that's pure entertainment .                                    │
├─────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│           0 │ they threw loads of money at an idea that should've been so much more even if it was  │
├─────────────┼───────────────────────────────────────────────────────────────────────────────────────┤
│           0 │ well-nigh unendurable . . . though